In [1]:
#setup - installing necessary packages
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
#from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import numpy as np

In [4]:
#import the data - generally in a semi-fixed state, but fixes can be made later if necessary
#note that the tilde is used as a delimiter because it is more rare in conversation than a comma
data = pd.read_csv('whatsapp_aru_fix.csv', header=0, delimiter="~", quoting=3)

In [5]:
#split initial data by person
jim= data[data['name']=='james e mooradian']
aru= data[data['name']=='Aru B']

In [6]:
#get rid of NA values
data_fix=data.dropna()

In [7]:
#import beautifulsoup and re for letters only
from bs4 import BeautifulSoup
import re


In [8]:
#import natural language tool kit and print stopwords to check that they're there

import nltk
from nltk.corpus import stopwords # Import the stop word list
#print stopwords.words("english")

In [9]:
#cleans up the data - lower case, remove nonletters, remove stopwords
#can specify other stopwords if necessary
def text_to_words(rtext, stops=stopwords.words('english')):
    #get text
    rtext2 = BeautifulSoup(rtext).get_text()
    #remove nonletters
    rtext3 = re.sub("[^a-zA-Z]", " ", rtext2) 
    #split by space, lowercase
    rtext4 = rtext3.lower().split()
    #define stopwords and remove stopwords
    stopword = stops
    rtext5 = [w for w in rtext4 if not w in stopword]
    #return as text joined by space - alternate, return as list of words
    return(" ".join(rtext5))
    #return rtext5

In [10]:
#define a full cleaning function

def bag_ready(inp_data,textval='text',stops=stopwords.words('english')):
    fix_text=[]
    last = inp_data.irow(0)
    for i in range(0, inp_data.shape[0]):
        fix_text.append(text_to_words(inp_data[textval].irow(i),stops))
        last = inp_data.irow(i)
    return fix_text

In [11]:
#get a rough 20% sample of the data
aru_samp=aru.sample(int(aru.shape[0])/5)
jim_samp=jim.sample(int(jim.shape[0])/5)


In [12]:
#append sample data and test shape
sample=aru_samp
sample=sample.append(jim_samp)
sample.shape

(3429, 7)

In [13]:
#get a set of sample rows and sort
samprows=sample.index
samprows=sorted(samprows, key=int)
trainrows = [w for w in data_fix.index if not w in samprows]

In [14]:
#length checks for sample and trian rows
print len(trainrows)
print len(samprows)
print data_fix.shape
print len(trainrows)+len(samprows)

13723
3429
(17152, 7)
17152


In [15]:
#subset the training and test sets and print shapes for checking
train = pd.DataFrame({'datetime':[],'name':[],'text':[]})
train = train.append(data_fix)
train = train.drop(samprows)
test = pd.DataFrame({'datetime':[],'name':[],'text':[]})
test = test.append(data_fix)
test = test.drop(trainrows)
print train.shape
print test.shape

(13723, 8)
(3429, 8)


In [16]:
#fix the text
fixed_text=bag_ready(train)

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: irow(i) is deprecated. Please use .iloc[i] or .iat[i]
C:\Program Files\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Program Files\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: irow(i) is deprecated.

In [17]:
#run the sklearn vectorizer on the fixed data (data_fix goes in)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

bag_of_words = vectorizer.fit_transform(fixed_text)
bag_of_words = bag_of_words.toarray()


In [18]:
#test for shape
bag_of_words.shape

(13723L, 5000L)

In [19]:
#make a categorical value out of the name values
namevals=train['name'].astype('category')


In [38]:
#train the random forest
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( bag_of_words, namevals )

Training the random forest...


In [39]:
#building test features
test_data_features = vectorizer.transform(bag_ready(test))
test_data_features = test_data_features.toarray()

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: irow(i) is deprecated. Please use .iloc[i] or .iat[i]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: irow(i) is deprecated. Please use .iloc[i]


In [40]:
result = forest.predict(test_data_features)

In [41]:
#output the test
pd.DataFrame(result).to_csv('result_output.csv')
test.to_csv('test_output.csv')
pd.DataFrame(bag_ready(data_fix)).to_csv('alldata_fix.csv')

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: irow(i) is deprecated. Please use .iloc[i] or .iat[i]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: irow(i) is deprecated. Please use .iloc[i]


In [42]:
#meant specifically for counting unique words
#cleans up the data - lower case, remove nonletters, remove stopwords
#can specify other stopwords if necessary
def text_to_words2(rtext, stops=stopwords.words('english')):
    #get text
    rtext2 = BeautifulSoup(rtext).get_text()
    #remove nonletters
    rtext3 = re.sub("[^a-zA-Z]", " ", rtext2) 
    #split by space, lowercase
    rtext4 = rtext3.lower().split()
    #define stopwords and remove stopwords
    stopword = stops
    rtext5 = [w for w in rtext4 if not w in stopword]
    #return as text joined by space - alternate, return as list of words
    #return(" ".join(rtext5))
    return rtext5

In [43]:
#appends text to words for all info

def bag_ready2(inp_data,textval='text',stops=stopwords.words('english')):
    fix_text=[]
    last = inp_data.irow(0)
    for i in range(0, inp_data.shape[0]):
        fix_text.append(text_to_words2(inp_data[textval].irow(i),stops))
        last = inp_data.irow(i)
    return fix_text

In [44]:
#flatten - gets all words to a single array
flatten = lambda l: [item for sublist in l for item in sublist]

In [45]:
#get all words for each person
atest=flatten(bag_ready2(aru))
jtest=flatten(bag_ready2(jim))
pd.DataFrame(atest).to_csv('aru_words.txt')
pd.DataFrame(jtest).to_csv('jim_wordsv2.txt')

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: irow(i) is deprecated. Please use .iloc[i] or .iat[i]
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: irow(i) is deprecated. Please use .iloc[i]


In [50]:
#get total words and output
alltxt=[]
alltxt.append(atest)
alltxt.append(jtest)
all_out=flatten(alltxt)
pd.DataFrame(alltxt).to_csv('all_words_achat.txt')

In [51]:
alltxt2 = " ".join(alltxt)
pd.DataFrame(alltxt).to_csv('all_words_achatv2.txt')

TypeError: sequence item 0: expected string, list found

In [53]:
from itertools import chain
alltxt2=" ".join(str(i) for i in chain(alltxt))
pd.DataFrame(alltxt2).to_csv('all_words_achatv2.txt')

PandasError: DataFrame constructor not properly called!

In [62]:
print alltxt2

[u'omg', u'baby', u'landed', u'work', u'isnt', u'yourope', u'amazing', u'love', u'want', u'move', u'u', u'drunk', u'already', u'lmao', u'also', u'proud', u'stay', u'till', u'like', u'aw', u'miss', u'much', u'fun', u'homes', u'bro', u'ooh', u'naps', u'eat', u'dinner', u'like', u'ridiculously', u'late', u'time', u'bit', u'hungover', u'day', u'lol', u'gonna', u'constantly', u'hungover', u'italy', u'lol', u'baby', u'man', u'want', u'sleep', u'booze', u'luckily', u'day', u'drinking', u'recovering', u'beaches', u'yay', u'currently', u'exhausted', u'need', u'good', u'nights', u'rest', u'tonight', u'baby', u'jealous', u'got', u'tiger', u'balmmmmm', u'saumil', u'massaged', u'back', u'almost', u'made', u'knot', u'go', u'away', u'magic', u'palm', u'lol', u'almost', u'normal', u'feels', u'good', u'aw', u'us', u'two', u'days', u'yayyy', u'prob', u'driving', u'scooters', u'around', u'town', u'text', u'hire', u'take', u'ferry', u'capri', u'take', u'high', u'speed', u'ferry', u'picks', u'separately', 

"[ u ' o m g ' ,   u ' b a b y ' ,   u ' l a n d e d ' ,   u ' w o r k ' ,   u ' i s n t ' ,   u ' y o u r o p e ' ,   u ' a m a z i n g ' ,   u ' l o v e ' ,   u ' w a n t ' ,   u ' m o v e ' ,   u ' u ' ,   u ' d r u n k ' ,   u ' a l r e a d y ' ,   u ' l m a o ' ,   u ' a l s o ' ,   u ' p r o u d ' ,   u ' s t a y ' ,   u ' t i l l ' ,   u ' l i k e ' ,   u ' a w ' ,   u ' m i s s ' ,   u ' m u c h ' ,   u ' f u n ' ,   u ' h o m e s ' ,   u ' b r o ' ,   u ' o o h ' ,   u ' n a p s ' ,   u ' e a t ' ,   u ' d i n n e r ' ,   u ' l i k e ' ,   u ' r i d i c u l o u s l y ' ,   u ' l a t e ' ,   u ' t i m e ' ,   u ' b i t ' ,   u ' h u n g o v e r ' ,   u ' d a y ' ,   u ' l o l ' ,   u ' g o n n a ' ,   u ' c o n s t a n t l y ' ,   u ' h u n g o v e r ' ,   u ' i t a l y ' ,   u ' l o l ' ,   u ' b a b y ' ,   u ' m a n ' ,   u ' w a n t ' ,   u ' s l e e p ' ,   u ' b o o z e ' ,   u ' l u c k i l y ' ,   u ' d a y ' ,   u ' d r i n k i n g ' ,   u ' r e c o v e r i n g ' ,   u